In [9]:
from data_generator import StudyParameters, TestConfiguration, create_test_instance
from experiment1_solution_quality import DynamicProgramming, DPState
import logging

In [10]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
def create_simple_test_instance():
    """Create a simple test instance with known parameters."""
    
    # Create basic parameters
    params = StudyParameters(
        T=2,          # 2 booking periods
        N=3,          # 3-day horizon
        C=5,          # 5 rooms capacity
        price_min=50, # Minimum price
        price_max=150,# Maximum price
        alpha=0.1,    # Smoothing parameters (not used in DP)
        beta=0.1      # Smoothing parameters (not used in DP)
    )
    
    # Generate all possible booking classes for 3-day horizon
    booking_classes = []
    for arrival in range(1, 4):  # 1 to 3
        for departure in range(arrival, 4):  # arrival to 3
            booking_classes.append((arrival, departure))
    
    # Create uniform arrival probabilities (0.15 for each booking class)
    arrival_probs = {}
    for t in range(1, 3):  # T=2 periods
        arrival_probs[t] = {bc: 0.15 for bc in booking_classes}
    
    # Create uniform price sensitivity (1/max_price for all classes)
    epsilon = 1/params.price_max
    price_sensitivity = {bc: epsilon for bc in booking_classes}
    
    # Create test instance
    test_instance = {
        'parameters': params,
        'booking_classes': booking_classes,
        'arrival_probabilities': arrival_probs,
        'reservation_price_params': price_sensitivity
    }
    
    return test_instance

In [12]:
def main():
    # Create test instance
    test_instance = create_simple_test_instance()
    
    # Print instance details for verification
    logger.info("Test Instance Details:")
    logger.info(f"Time Periods (T): {test_instance['parameters'].T}")
    logger.info(f"Service Horizon (N): {test_instance['parameters'].N}")
    logger.info(f"Room Capacity (C): {test_instance['parameters'].C}")
    logger.info(f"Price Range: [{test_instance['parameters'].price_min}, {test_instance['parameters'].price_max}]")
    
    logger.info("\nBooking Classes:")
    for bc in test_instance['booking_classes']:
        logger.info(f"Class {bc}: Length of stay = {bc[1] - bc[0] + 1}")
    
    logger.info("\nArrival Probabilities (sample):")
    for bc, prob in test_instance['arrival_probabilities'][1].items():
        logger.info(f"Class {bc}: {prob}")
    
    logger.info("\nPrice Sensitivity:")
    sensitivity = next(iter(test_instance['reservation_price_params'].values()))
    logger.info(f"Uniform epsilon: {sensitivity}")
    
    # Solve using Dynamic Programming
    dp = DynamicProgramming(test_instance)
    optimal_value, optimal_policy = dp.solve()
    
    logger.info("\nResults:")
    logger.info(f"Optimal Expected Revenue: {optimal_value:.2f}")
    
    # Print optimal prices for initial state
    initial_state = DPState(
        capacity=tuple(test_instance['parameters'].C for _ in range(test_instance['parameters'].N)),
        time=1
    )
    
    if initial_state in optimal_policy:
        logger.info("\nOptimal Initial Prices:")
        prices = optimal_policy[initial_state]
        for day, price in enumerate(prices, 1):
            logger.info(f"Day {day}: ${price:.2f}")

In [13]:
if __name__ == "__main__":
    main()

INFO:__main__:Test Instance Details:
INFO:__main__:Time Periods (T): 2
INFO:__main__:Service Horizon (N): 3
INFO:__main__:Room Capacity (C): 5
INFO:__main__:Price Range: [50, 150]
INFO:__main__:
Booking Classes:
INFO:__main__:Class (1, 1): Length of stay = 1
INFO:__main__:Class (1, 2): Length of stay = 2
INFO:__main__:Class (1, 3): Length of stay = 3
INFO:__main__:Class (2, 2): Length of stay = 1
INFO:__main__:Class (2, 3): Length of stay = 2
INFO:__main__:Class (3, 3): Length of stay = 1
INFO:__main__:
Arrival Probabilities (sample):
INFO:__main__:Class (1, 1): 0.15
INFO:__main__:Class (1, 2): 0.15
INFO:__main__:Class (1, 3): 0.15
INFO:__main__:Class (2, 2): 0.15
INFO:__main__:Class (2, 3): 0.15
INFO:__main__:Class (3, 3): 0.15
INFO:__main__:
Price Sensitivity:
INFO:__main__:Uniform epsilon: 0.006666666666666667
INFO:experiment1_solution_quality:Generated 27 price combinations
INFO:experiment1_solution_quality:Initialized DP solver with T=2, N=3, C=5
INFO:experiment1_solution_quality: